In [ ]:
import plotting_functions as pf
import pandas as pd
import numpy as np

clk = "5.00"
freq = "1GHz" if clk == "1.00" else "200MHz"
export = f"../results/breakdown"
prism_prune = [
    "#5F4690",
    "#38A6A5",
    "#0F8554",
    "#EDAD08",
    "#CC503E",
    "#94346E",
    "#d68bd2",
    "#666666",
]

In [ ]:
clock = float(clk)
DIR_FU = f"../results/breakdown/{clk}/FU"
DIR_SWU = f"../results/breakdown/{clk}/SWU"
df_FU = pd.read_csv(f"{DIR_FU}/power.csv", index_col=[0, 1])
df_SWU = pd.read_csv(f"{DIR_SWU}/power.csv", index_col=[0, 1])

df_FU.loc["8x8"] = df_FU.loc["8x8"].mul(clock * (10 ** 3) / 512).values
df_FU.loc["8x4"] = df_FU.loc["8x4"].mul(clock * (10 ** 3) / 1024).values
df_FU.loc["8x2"] = df_FU.loc["8x2"].mul(clock * (10 ** 3) / 2048).values
df_FU.loc["4x4"] = df_FU.loc["4x4"].mul(clock * (10 ** 3) / 2048).values
df_FU.loc["2x2"] = df_FU.loc["2x2"].mul(clock * (10 ** 3) / 8192).values

df_SWU.loc["8x8"] = df_SWU.loc["8x8"].mul(clock * (10 ** 3) / 512).values
df_SWU.loc["4x4"] = df_SWU.loc["4x4"].mul(clock * (10 ** 3) / 1024).values
df_SWU.loc["2x2"] = df_SWU.loc["2x2"].mul(clock * (10 ** 3) / 2048).values
for prec in ["8x4", "8x2"]:
    for row in df_SWU.index.levels[1]:
        df_SWU.loc[(prec, row), :] = 0

df_area_FU = pd.read_csv(f"{DIR_FU}/area.csv", index_col=0)
df_area_SWU = pd.read_csv(f"{DIR_SWU}/area.csv", index_col=0)

In [ ]:
df_area = pd.concat([df_area_FU, df_area_SWU])
df_area.rename(
    index={
        "BITFUSION": "BG_L2_L4_00_L3_11_L2_11_DVAFS_0",
        "BITBLADE": "BG_L3_L4_00_L3_11_L2_11_DVAFS_0",
        "LOOM": "BG_BS_L4_00_L3_00_L2_11_DVAFS_0",
    },
    columns={"comb": "Combinational", "seq": "Sequential"},
    inplace=True,
)
df_area.index.set_names("design", inplace=True)
df_area = df_area / 1e6

df_energy = pd.concat([df_FU, df_SWU])
df_energy.index.set_names(("prec", "design"), inplace=True)
ordered_list = [
    "FU\nBG: L2\nL2: IS",
    "FU\nBG: L2\nL2: HS",
    "FU\nBG: L2\nL2: OS",
    "FU\nBG: L3\nL2: HS",
    "FU\nBG: L3\nL2: OS",
    "FU\nBG: BS\nL2: OS",
    "SWU\nBG: L2\nL2: NO",
    "SWU\nBG: L2\nL2: OS",
]
col_dict = {
    "mult_2x2": "L1 Multipliers",
    "L2_tree": "L2 Adder Tree",
    "L3_tree": "L3 Adder Tree",
    "L4_tree": "L4 Adder Tree",
    "out_reg": "Output Registers",
    "pipe_reg": "Internal Registers",
    "in_reg": "Input Registers",
    "accum": "Output Accumulators",
}
df_energy.rename(
    index={
        "BITFUSION": "BG_L2_L4_00_L3_11_L2_11_DVAFS_0",
        "BITBLADE": "BG_L3_L4_00_L3_11_L2_11_DVAFS_0",
        "LOOM": "BG_BS_L4_00_L3_00_L2_11_DVAFS_0",
    },
    columns=col_dict,
    inplace=True,
)


In [ ]:
df_scatter = pd.DataFrame("0", index=df_energy.index, columns=["Energy/Op", "Area"])
for prec in ["8x8", "8x4", "8x2", "4x4", "2x2"]:
    df_scatter.loc[prec] = pd.concat(
        [df_energy.loc[prec]["top"], df_area["top"]], axis=1
    ).values
df_scatter.reset_index(level="design", inplace=True)
df_scatter["Config / BG / L2"] = df_scatter["design"].apply(pf.SWPBGL2_noline)
df_scatter["L4 / L3 Modes"] = df_scatter["design"].apply(pf.L4L3)
df_scatter["SWP"] = df_scatter["design"].apply(pf.SWP)
df_scatter.set_index([df_scatter.index, "design"], inplace=True)
df_scatter.rename(
    index={
        "8x8": "8b x 8b",
        "8x4": "8b x 4b",
        "8x2": "8b x 2b",
        "4x4": "4b x 4b",
        "2x2": "2b x 2b",
    },
    inplace=True,
)
palette = {
    "FU / BG: L2 / L2: IS": "tab:blue",
    "FU / BG: L2 / L2: HS": "tab:green",
    "FU / BG: L2 / L2: OS": "tab:red",
    "FU / BG: L3 / L2: HS": "tab:brown",
    "FU / BG: L3 / L2: OS": "tab:orange",
    "FU / BG: BS / L2: OS": "tab:cyan",
    "SWU / BG: L2 / L2: NO": "tab:pink",
    "SWU / BG: L2 / L2: OS": "tab:purple",
}

In [ ]:
df_energy.reset_index(level="design", inplace=True)
df_energy["BG Unrolling / L2 Mode"] = df_energy["design"].apply(pf.SWPBGL2)
df_energy["L4 / L3 Modes"] = df_energy["design"].apply(pf.L4L3)
df_energy.drop(labels="design", axis="columns", inplace=True)
df_energy.set_index(
    [df_energy.index, "BG Unrolling / L2 Mode", "L4 / L3 Modes"], inplace=True
)
df_energy = df_energy.reindex(ordered_list, level=1)

df_area.reset_index(level="design", inplace=True)
df_area["BG Unrolling / L2 Mode"] = df_area["design"].apply(pf.SWPBGL2)
df_area["L4 / L3 Modes"] = df_area["design"].apply(pf.L4L3)
df_area.drop(labels="design", axis="columns", inplace=True)
df_area.set_index(["BG Unrolling / L2 Mode", "L4 / L3 Modes"], inplace=True)
df_area = df_area.reindex(ordered_list, level=0)

In [ ]:
BAR_AREA = ["Combinational", "Sequential"]
pf.plot_clustered_stacked(
    df_area[BAR_AREA],
    title=f"Area_Breakdown",
    save=True,
    export=f"{export}/{clk}",
    ext="png",
    cmap="tab20b",
    width=1,
)

In [ ]:
# In the paper, there was no room to show all 5 tested precisions
# We opted to drop 8b x 2b precision, and show the rest
# This is optional, you can still plot 8x2 precision
pf.scatter_extract(
    df_scatter.drop("8b x 2b", level="prec").replace(0, np.nan).dropna(),
    name="alpha",
    ext="png",
    export=f"{export}/{clk}",
    legend=(freq == "1GHz"),
    palette=palette,
)

In [ ]:
BAR_POWER = [
    "L1 Multipliers",
    "L2 Adder Tree",
    "L3 Adder Tree",
    "L4 Adder Tree",
    "Output Registers",
    "Input Registers",
    "Internal Registers",
    "Output Accumulators",
]
for prec in [
    "8x8",
    "8x4",
    "8x2",
    "4x4",
    "2x2",
]:
    pf.plot_clustered_stacked(
        df_energy.xs(prec)[BAR_POWER],
        title=f"Energy_{prec}".replace(" ", "").replace("b", ""),
        save=True,
        export=f"{export}/{clk}",
        ext="png",
        color=prism_prune,
        width=1,
        sep_legend=True,
    )

In [ ]:
df_sq_5_FU = pf.energy_extract("5.00", False)
df_sq_1_FU = pf.energy_extract("1.00", False)
df_sq_5_SWU = pf.energy_extract("5.00", True)
df_sq_1_SWU = pf.energy_extract("1.00", True)
df_sq_5_total = pd.concat([df_sq_5_FU, df_sq_5_SWU], axis=1, join="inner")
df_sq_1_total = pd.concat([df_sq_1_FU, df_sq_1_SWU], axis=1, join="inner")

In [ ]:
df_sq_5_total.rename(
    index={
        "8x8": "8b x 8b",
        "8x4": "8b x 4b",
        "8x2": "8b x 2b",
        "4x4": "4b x 4b",
        "2x2": "2b x 2b",
    }, 
    inplace=True
)
df_sq_1_total.rename(
    index={
        "8x8": "8b x 8b",
        "8x4": "8b x 4b",
        "8x2": "8b x 2b",
        "4x4": "4b x 4b",
        "2x2": "2b x 2b",
    }, 
    inplace=True
)

In [ ]:
# ideal_loops = {"B": 1, "K": 64, "C": 256, "OY": 8, "OX": 8, "FY": 4, "FX": 4}
# realistic_loops = {"B": 1, "K": 64, "C": 256, "OY": 7, "OX": 7, "FY": 3, "FX": 3}
# df_util = pf.square_util(realistic_loops)
# df_util_5 = df_sq_5 * df_util
# df_util_1 = df_sq_1 * df_util
# df_mix_5 = (
#     df_sq_5.xs("8x8") * 0.2
#     + df_sq_5.xs("8x4") * 0.15
#     + df_sq_5.xs("8x2") * 0.15
#     + df_sq_5.xs("4x4") * 0.3
#     + df_sq_5.xs("2x2") * 0.2
# )
# df_mix_1 = (
#     df_sq_1.xs("8x8") * 0.2
#     + df_sq_1.xs("8x4") * 0.15
#     + df_sq_1.xs("8x2") * 0.15
#     + df_sq_1.xs("4x4") * 0.3
#     + df_sq_1.xs("2x2") * 0.2
# )
# df_mix = pd.concat([df_mix_5, df_mix_1], keys=["200 MHz", "1 GHz"], names=["freq"])


In [ ]:
DVAFS = False
pf.heatmap_extract(
    df_sq=df_sq_5_total.drop("8b x 2b", level="prec"),
    name="total",
    cmap="viridis_r",
    ext="png",
    export=f"{export}/5.00",
    DVAFS=DVAFS,
)
pf.heatmap_extract(
    df_sq=df_sq_1_total.drop("8b x 2b", level="prec"),
    name="total",
    cmap="viridis_r",
    ext="png",
    export=f"{export}/1.00",
    DVAFS=DVAFS,
    ylabels=False,
)